In [2]:
import warnings
warnings.filterwarnings('ignore')
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
import time
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from ploting import *
%matplotlib inline

In [3]:
def normalize(data):
    row_sums = data.sum(axis=1)
    new_matrix = data / row_sums[:, np.newaxis]
    return new_matrix

In [4]:
latexify(fig_width=6)

## Load data

In [4]:
data= pd.read_csv("data/admission.txt",  header=None)
X = data.iloc[:,0:2].as_matrix()
y = data.iloc[:,-1].as_matrix().reshape(-1,1)
X_normal = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X_normal, y, test_size=0.10, random_state=42)

## Define model

In [5]:
class Model(torch.nn.Module):

    def __init__(self, nb_input, nb_output):
        """
        In the constructor we instantiate two nn.Linear module
        """
        super(Model, self).__init__()
        
        self.l1 = torch.nn.Linear(nb_input, 6)
        self.l2 = torch.nn.Linear(6, 4)
        self.l3 = torch.nn.Linear(4, nb_output)
        
        self.sigmoid = torch.nn.Sigmoid()
        self.relu = torch.nn.ReLU()
        

    def forward(self, x):
        """
        In the forward function we accept a Variable of input data and we must return
        a Variable of output data. We can use Modules defined in the constructor as
        well as arbitrary operators on Variables.
        """
        out1 = self.relu(self.l1(x))
        out2 = self.relu(self.l2(out1))
        y_pred = self.sigmoid(self.l3(out2))
        return y_pred

## Define prediction

In [6]:
def predict(x_data, model):
    x_data = torch.from_numpy(x_data).float()
    x_data = Variable(x_data)
    
    if torch.cuda.is_available():
        model = model.cuda()
        x_data = x_data.cuda()
        
    y_pred = model(x_data) 
    
    if torch.cuda.is_available():
        y_pred=y_pred.cpu()
        
         
    return  y_pred.data.numpy()

## Define training function

In [7]:
def model_train(x_data, y_data, model, lrate):
    
    x_data = torch.from_numpy(x_data).float()
    y_data = torch.from_numpy(y_data).float()
    
    x_data = Variable(x_data)
    y_data = Variable(y_data)
    
    if torch.cuda.is_available():
        model = model.cuda()
        x_data = x_data.cuda()
        y_data = y_data.cuda()
        
    criterion = torch.nn.BCELoss(size_average=True)
    optimizer = torch.optim.SGD(model.parameters(), lr=lrate)
    
    
    Epoch = []
    Loss  = []
    start_time = time.time()
    
    for epoch in range(1000):
        y_pred = model(x_data)

        loss = criterion(y_pred, y_data)
        Epoch.append(epoch)
        Loss.append(loss.data[0])
    
        # Zero gradients, perform a backward pass, and update the weights.
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    
    return Epoch, Loss 

In [8]:
nb_input = X_train.shape[1]
nb_output = y_train.shape[1]
nb_hidden = 5
model = Model(nb_input, nb_output)

In [9]:
print(model)

Model(
  (l1): Linear(in_features=2, out_features=6)
  (l2): Linear(in_features=6, out_features=4)
  (l3): Linear(in_features=4, out_features=1)
  (sigmoid): Sigmoid()
  (relu): ReLU()
)


In [10]:
epoch, loss = model_train(X_train, y_train, model, 0.1)

In [11]:
plt.plot(epoch, loss, label="$lrate=0.1$")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend();

In [ ]:
pred = predict(X_test, model) 
acc = accuracy_score(y_test, np.around(pred))
fscore = f1_score(y_test, np.around(pred))
print("Model accuracy is {} %".format(acc*100))
print("Model F-Measure is {} %".format(fscore))